In [204]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import randint, uniform
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import math
from category_encoders import OrdinalEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, classification_report, roc_curve, ConfusionMatrixDisplay, roc_auc_score, accuracy_score, plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings("ignore")
import eli5
from eli5.sklearn import PermutationImportance

games = 게임날짜,게임ID,게임상황,홈팀 ID,어웨이팀 ID,시즌,홈팀 ID,홈팀 득점,홈팀 필드골 성공율,홈팀 자유투 성공율, 홈팀 3점 성공율,홈팀 어시스트,홈팀 리바운드,어웨이팀 ID,어웨이팀 득점,어웨이팀 필드골 성공율, 어웨이팀 자유투 성공율, 어웨이팀 3점 성공율,어웨이팀,어시스트,어웨이팀 리바운드,경기결과

games_details = 게임ID,팀 ID,팀 약어,팀 연고지,선수 ID,선수 이름,스타팅포지션,코멘트(출전하지 않는 이유,부상 등),출전시간,필드골 성공,필드골 시도,필드골 비율,3점 성공,3점 시도,3점 비율,자유투 성공,자유투 시도,자유투 비율,공격리바운드,수비리바운드,총 리바운드,어시스트,스틸,블락 샷,턴오버,개인파울,총 득점,득점마진

players = 선수이름,팀 ID,선수 ID,시즌

ranking = 팀 ID,리그 ID (Only NBA),시즌 ID,게임날짜,컨퍼런스 (서부,동부),팀 이름,이번 시즌 게임 수,이번 시즌 승리 수,이번 시즌 패배 수,승리 비율,홈 경기 누적 결과,어웨이 경기 누적 결과,재경기

teams = 리그 ID,팀 ID,최초 NBA 합류,최대 NBA 합류,약어,닉네임,창설연도,도시,경기장 이름,경기장 최대 수용인원,구단주,매니저,코치,리그제휴

In [205]:
df_team = pd.read_csv('teams.csv') 
df_team = df_team[['TEAM_ID', 'ABBREVIATION']] # 팀 약어 추출

In [206]:
df = pd.read_csv('games_details.csv')
df1 = df[df['TEAM_ID']==1610612744]
df2 = df1[df1['GAME_ID'].between(22100001,22101230)]
df_curry = df2[df2['PLAYER_ID']==201939]

In [207]:
df_game = pd.read_csv('games.csv')
df_game2 = df_game[df_game['GAME_ID'].between(22100001,22101230)]
df_game3 = df_game2[(df_game2['HOME_TEAM_ID']==1610612744)|(df_game2['VISITOR_TEAM_ID']==1610612744)]
conditionlist = [
    (df_game3['HOME_TEAM_ID'] ==1610612744) & (df_game3['HOME_TEAM_WINS']==1) ,
    (df_game3['VISITOR_TEAM_ID'] ==1610612744) & (df_game3['HOME_TEAM_WINS']==0)]
choicelist = ['1', '1']
df_game3['GWS_WIN'] = np.select(conditionlist, choicelist, default='0')

df_GWSWINgame3 = df_game3[['GAME_ID','GWS_WIN']]

In [208]:
df_curry_WL=pd.merge(left = df_curry , right = df_GWSWINgame3, how = "left", on = "GAME_ID")

In [209]:
#,'MIN'
df_curry_WL = df_curry_WL[['TEAM_ABBREVIATION',
       'PLAYER_NAME', 'FGM','FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS',
       'PLUS_MINUS', 'GWS_WIN']]

In [210]:
df_curry_WL

,TEAM_ABBREVIATION,PLAYER_NAME,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,...,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,GWS_WIN
0,GSW,Stephen Curry,1.0,4.0,0.250,1.0,4.0,0.250,0.0,0.0,...,1.0,1.0,2.0,0.0,0.0,4.0,2.0,3.0,-5.0,0
1,GSW,Stephen Curry,16.0,25.0,0.640,7.0,14.0,0.500,8.0,8.0,...,6.0,6.0,6.0,1.0,0.0,2.0,1.0,47.0,17.0,1
2,GSW,Stephen Curry,3.0,7.0,0.429,2.0,4.0,0.500,0.0,0.0,...,4.0,5.0,8.0,1.0,1.0,4.0,1.0,8.0,14.0,1
3,GSW,Stephen Curry,11.0,21.0,0.524,5.0,12.0,0.417,7.0,7.0,...,9.0,9.0,3.0,2.0,0.0,2.0,1.0,34.0,18.0,1
4,GSW,Stephen Curry,5.0,12.0,0.417,1.0,6.0,0.167,4.0,4.0,...,5.0,5.0,5.0,3.0,0.0,4.0,4.0,15.0,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,GSW,Stephen Curry,11.0,29.0,0.379,7.0,20.0,0.350,7.0,7.0,...,6.0,7.0,8.0,1.0,0.0,5.0,2.0,36.0,12.0,0
60,GSW,Stephen Curry,6.0,14.0,0.429,4.0,9.0,0.444,7.0,7.0,...,5.0,6.0,4.0,0.0,1.0,4.0,1.0,23.0,15.0,1
61,GSW,Stephen Curry,9.0,23.0,0.391,4.0,15.0,0.267,5.0,6.0,...,6.0,7.0,10.0,3.0,0.0,3.0,1.0,27.0,14.0,1
62,GSW,Stephen Curry,16.0,25.0,0.640,8.0,13.0,0.615,5.0,5.0,...,10.0,10.0,1.0,1.0,1.0,6.0,2.0,45.0,-2.0,1


In [211]:
target = 'GWS_WIN'

In [212]:
train, val = train_test_split(df_curry_WL, train_size=0.80, test_size=0.20, stratify=df_curry_WL[target], random_state=2)

In [213]:
features = train.drop(columns=[target]).columns

In [214]:
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]

In [215]:
pipe = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(n_estimators=100, random_state=2, n_jobs=-1)
)
dists = {
    'simpleimputer__strategy': ['mean', 'median', 'most_frequent'], 
    'randomforestclassifier__n_estimators': randint(50, 500), 
    'randomforestclassifier__max_depth': [5, 10, 15, 20, None], 
    'randomforestclassifier__max_features': uniform(0, 1)
}
clf = RandomizedSearchCV(
    pipe, 
    param_distributions=dists, 
    n_iter=50, 
    cv=3, 
    scoring='accuracy',  
    verbose=1,
    n_jobs=-1
)
clf.fit(X_train, y_train);
print('최적 하이퍼파라미터: ', clf.best_params_)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
최적 하이퍼파라미터:  {'randomforestclassifier__max_depth': 5, 'randomforestclassifier__max_features': 0.8667648772388833, 'randomforestclassifier__n_estimators': 217, 'simpleimputer__strategy': 'mean'}


In [225]:
pipe_selected = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(strategy='mean'),
    RandomForestClassifier(random_state=2, max_depth=5, max_features=0.8667648772388833, n_estimators= 217)
)

pipe_selected.fit(X_train, y_train)

y_pred = pipe_selected.predict(X_val)
pd.DataFrame(clf.cv_results_).sort_values(by='rank_test_score').T

,6,24,25,11,39,4,5,26,46,23,...,16,7,21,40,14,33,41,9,3,1
mean_fit_time,0.856332,1.174,0.969,1.204667,0.481667,1.63,0.753,1.523666,1.199333,0.450668,...,0.280999,1.518669,1.589002,1.542334,1.120997,0.47,0.902667,1.027334,0.767999,1.076666
std_fit_time,0.067058,0.008641,0.042551,0.020531,0.031847,0.127598,0.084762,0.051913,0.036114,0.005736,...,0.011774,0.199016,0.042832,0.053018,0.02581,0.011517,0.020271,0.119291,0.068065,0.079486
mean_score_time,0.2,0.153001,0.131667,0.133334,0.092,0.310001,0.136667,0.176,0.117334,0.079333,...,0.096334,0.246,0.225667,0.181333,0.143667,0.083,0.118667,0.132332,0.165001,0.162335
std_score_time,0.068446,0.003741,0.004496,0.001699,0.014989,0.10288,0.04485,0.004968,0.004784,0.000943,...,0.013768,0.101123,0.073292,0.015369,0.009288,0.003559,0.004189,0.014703,0.050165,0.038665
param_randomforestclassifier__max_depth,5,None,15,None,10,5,10,20,5,20,...,15,5,15,15,20,10,5,15,None,20
param_randomforestclassifier__max_features,0.866765,0.955824,0.980333,0.980482,0.689425,0.975499,0.489423,0.48622,0.924646,0.754885,...,0.00327,0.05169,0.005884,0.013902,0.003149,0.023321,0.0586,0.053447,0.034463,0.092972
param_randomforestclassifier__n_estimators,217,320,261,278,128,427,195,461,367,124,...,64,404,473,459,337,133,263,288,182,284
param_simpleimputer__strategy,mean,most_frequent,median,most_frequent,mean,mean,median,most_frequent,most_frequent,most_frequent,...,most_frequent,median,median,median,median,median,mean,median,mean,most_frequent
params,"{'randomforestclassifier__max_depth': 5, 'rand...","{'randomforestclassifier__max_depth': None, 'r...","{'randomforestclassifier__max_depth': 15, 'ran...","{'randomforestclassifier__max_depth': None, 'r...","{'randomforestclassifier__max_depth': 10, 'ran...","{'randomforestclassifier__max_depth': 5, 'rand...","{'randomforestclassifier__max_depth': 10, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 5, 'rand...","{'randomforestclassifier__max_depth': 20, 'ran...",...,"{'randomforestclassifier__max_depth': 15, 'ran...","{'randomforestclassifier__max_depth': 5, 'rand...","{'randomforestclassifier__max_depth': 15, 'ran...","{'randomforestclassifier__max_depth': 15, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 10, 'ran...","{'randomforestclassifier__max_depth': 5, 'rand...","{'randomforestclassifier__max_depth': 15, 'ran...","{'randomforestclassifier__max_depth': None, 'r...","{'randomforestclassifier__max_depth': 20, 'ran..."
split0_test_score,0.882353,0.823529,0.823529,0.823529,0.823529,0.823529,0.823529,0.823529,0.823529,0.823529,...,0.647059,0.705882,0.705882,0.705882,0.647059,0.647059,0.647059,0.647059,0.647059,0.647059


In [226]:
df_test = pd.read_csv('games_details.csv')
df1_test = df_test[df_test['TEAM_ID']==1610612744]
df2_test = df1_test[df1_test['GAME_ID'].between(22200001,22200470)]
df_curry_test = df2_test[df2_test['PLAYER_ID']==201939]

In [227]:
df_game_test = pd.read_csv('games.csv')
df_game2_test = df_game_test[df_game_test['GAME_ID'].between(22200001,22200470)]
df_game3_test = df_game2_test[(df_game2_test['HOME_TEAM_ID']==1610612744)|(df_game2_test['VISITOR_TEAM_ID']==1610612744)]
conditionlist_test = [
    (df_game3_test['HOME_TEAM_ID'] ==1610612744) & (df_game3_test['HOME_TEAM_WINS']==1) ,
    (df_game3_test['VISITOR_TEAM_ID'] ==1610612744) & (df_game3_test['HOME_TEAM_WINS']==0)]
choicelist = ['1', '1']
df_game3_test['GWS_WIN'] = np.select(conditionlist_test, choicelist, default='0')

df_GWSWINgame3_test = df_game3_test[['GAME_ID','GWS_WIN']]

In [228]:
df_curry_WL_test=pd.merge(left = df_curry_test , right = df_GWSWINgame3_test, how = "left", on = "GAME_ID")

In [229]:
df_curry_WL_test = df_curry_WL_test[['TEAM_ABBREVIATION',
       'PLAYER_NAME', 'FGM','FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS',
       'PLUS_MINUS', 'GWS_WIN']]

In [230]:
df_curry_WL_test

,TEAM_ABBREVIATION,PLAYER_NAME,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,...,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,GWS_WIN
0,GSW,Stephen Curry,11.0,19.0,0.579,5.0,10.0,0.500,11.0,11.0,...,6.0,7.0,7.0,2.0,1.0,3.0,2.0,38.0,-6.0,0
1,GSW,Stephen Curry,6.0,17.0,0.353,3.0,10.0,0.300,5.0,5.0,...,4.0,6.0,1.0,1.0,1.0,2.0,0.0,20.0,-20.0,0
2,GSW,Stephen Curry,12.0,21.0,0.571,6.0,11.0,0.545,2.0,2.0,...,5.0,6.0,7.0,0.0,1.0,4.0,3.0,32.0,20.0,1
3,GSW,Stephen Curry,3.0,17.0,0.176,2.0,10.0,0.200,4.0,4.0,...,4.0,4.0,6.0,1.0,0.0,3.0,2.0,12.0,-17.0,0
4,GSW,Stephen Curry,10.0,21.0,0.476,8.0,17.0,0.471,2.0,3.0,...,3.0,3.0,10.0,1.0,1.0,4.0,0.0,30.0,19.0,1
5,GSW,Stephen Curry,6.0,16.0,0.375,4.0,8.0,0.500,3.0,3.0,...,11.0,11.0,6.0,0.0,1.0,6.0,5.0,19.0,-1.0,1
6,GSW,Stephen Curry,11.0,24.0,0.458,5.0,14.0,0.357,5.0,6.0,...,4.0,5.0,5.0,0.0,0.0,4.0,4.0,32.0,-12.0,0
7,GSW,Stephen Curry,7.0,17.0,0.412,4.0,11.0,0.364,7.0,7.0,...,10.0,11.0,8.0,1.0,0.0,7.0,1.0,25.0,22.0,1
8,GSW,Stephen Curry,13.0,23.0,0.565,6.0,13.0,0.462,1.0,1.0,...,5.0,5.0,4.0,2.0,0.0,5.0,4.0,33.0,19.0,1
9,GSW,Stephen Curry,9.0,20.0,0.450,4.0,10.0,0.400,0.0,0.0,...,5.0,6.0,9.0,0.0,0.0,1.0,0.0,22.0,18.0,1


In [231]:
X_test = df_curry_WL_test[features]

In [253]:
X_test_20 = X_test.loc[[20]]
X_test_20
y_20_pred = pipe_selected.predict(X_test_20)
print('stephen curry 작년 승률에 의한 이번년도 20번째 경기 승패예측:', y_20_pred)
y_20_pred_proba = pipe_selected.predict_proba(X_test_20)[:, 1]
print('20번째 경기 승리 확률: ', y_20_pred_proba)

stephen curry 작년 승률에 의한 이번년도 20번째 경기 승패예측: ['0']
20번째 경기 승리 확률:  [0.35483871]


In [251]:
y_pred_proba = pipe_selected.predict_proba(X_test)[:, 1]
y_pred_proba

array([0.13364055, 0.1843318 , 0.98156682, 0.15668203, 0.93087558,
       0.62211982, 0.0921659 , 0.97235023, 1.        , 0.96774194,
       0.90322581, 0.9124424 , 0.07373272, 0.96313364, 0.40552995,
       0.96313364, 0.97235023, 0.93087558, 0.91705069, 0.0875576 ,
       0.35483871, 0.97695853, 0.13364055, 0.99078341, 0.89861751,
       0.91705069])

In [252]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

ValueError: Found input variables with inconsistent numbers of samples: [7, 26]